In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Deconvolution2D, MaxPooling2D, Flatten
from keras import layers as L

Using TensorFlow backend.


In [2]:
pwd

'C:\\Users\\david\\Desktop\\DATAMAD-0819\\laboratorios\\Final_Project\\Final_Project\\SRC'

In [3]:
path = './Euro_Sat/'

In [4]:
dic = {}
entrada = []
salida = []
i = 0

for ele in os.listdir(path):
    if os.path.isdir(path+ele):
        for file in os.listdir(path+ele):
            im = Image.open(os.path.join(path,ele,file))
            im_array = np.array(im)
            entrada.append(im_array)
            salida.append(i)
        dic[i] = ele
        i += 1


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: './Euro_Sat/'

In [ ]:
clases_salida = to_categorical(salida)

In [ ]:
entrada = np.array(entrada)
entrada = entrada/255

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(entrada, clases_salida, test_size=0.2, shuffle=True)

In [ ]:
in_shape = x_train[0,:,:,:].shape

model = Sequential()

model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(50))
model.add(Dense(len(dic), activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

In [ ]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# serialize model to JSON
model_1 = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_1)

In [ ]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [82]:
prueba_test = model.predict(x_test)

In [83]:
x_test

array([[[[0.58431373, 0.50588235, 0.47843137],
         [0.58039216, 0.50588235, 0.48235294],
         [0.48235294, 0.41960784, 0.42352941],
         ...,
         [0.48627451, 0.43529412, 0.4       ],
         [0.48235294, 0.41568627, 0.38823529],
         [0.54117647, 0.4745098 , 0.44705882]],

        [[0.58039216, 0.50588235, 0.49019608],
         [0.57647059, 0.50588235, 0.49803922],
         [0.4745098 , 0.41960784, 0.41960784],
         ...,
         [0.48627451, 0.43529412, 0.40392157],
         [0.48235294, 0.41568627, 0.38823529],
         [0.54509804, 0.47058824, 0.44705882]],

        [[0.51764706, 0.4627451 , 0.4627451 ],
         [0.50980392, 0.4627451 , 0.4627451 ],
         [0.4745098 , 0.42745098, 0.42745098],
         ...,
         [0.47843137, 0.42745098, 0.40392157],
         [0.47843137, 0.40392157, 0.38039216],
         [0.6       , 0.52156863, 0.49411765]],

        ...,

        [[0.79607843, 0.58823529, 0.53333333],
         [0.79215686, 0.58823529, 0.5372549 ]

In [103]:
pred_test = [dic[np.argmax(p)] for p in prueba_test]

In [104]:
suelos_test = [dic[np.argmax(s)] for s in y_test]

In [105]:
s = suelos_test[:15]

In [106]:
t = pred_test[:15]

In [107]:
import pandas as pd

d = {'y_Actual': s,
        'y_Predicted': t}

dat = pd.DataFrame(data=d)
dat

,y_Actual,y_Predicted
0,PermanentCrop,PermanentCrop
1,PermanentCrop,PermanentCrop
2,Industrial,Industrial
3,Forest,Forest
4,Forest,Pasture
5,River,River
6,Residential,Residential
7,Residential,Residential
8,AnnualCrop,Pasture
9,Forest,Forest


In [108]:
from sklearn.metrics import confusion_matrix
confusion_matrix(s, t)

array([[1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 2, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

In [109]:
my_image_resized = plt.imread("test_sea.jpg")

In [110]:
import numpy as np
probabilities = model.predict(np.array([my_image_resized,]))

In [111]:
probabilities

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [112]:
number_to_class = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
                   'Pasture', 'PermanentCrop', 'Residential', 'river', 'seaLake']

In [101]:
index = np.argsort(probabilities[0,:])

In [102]:
print("Most likely class:", number_to_class[index[9]], "-- Probability:", probabilities[0,index[9]])
print("Second most likely class:", number_to_class[index[8]], "-- Probability:", probabilities[0,index[8]])
print("Third most likely class:", number_to_class[index[7]], "-- Probability:", probabilities[0,index[7]])
print("Fourth most likely class:", number_to_class[index[6]], "-- Probability:", probabilities[0,index[6]])
print("Fifth most likely class:", number_to_class[index[5]], "-- Probability:", probabilities[0,index[5]])
print("sixth most likely class:", number_to_class[index[9]], "-- Probability:", probabilities[0,index[4]])
print("seventh most likely class:", number_to_class[index[8]], "-- Probability:", probabilities[0,index[3]])
print("eight most likely class:", number_to_class[index[7]], "-- Probability:", probabilities[0,index[2]])
print("nineth most likely class:", number_to_class[index[6]], "-- Probability:", probabilities[0,index[1]])
print("tenth most likely class:", number_to_class[index[5]], "-- Probability:", probabilities[0,index[0]])

Most likely class: Industrial -- Probability: 1.0
Second most likely class: seaLake -- Probability: 0.0
Third most likely class: river -- Probability: 0.0
Fourth most likely class: Residential -- Probability: 0.0
Fifth most likely class: PermanentCrop -- Probability: 0.0
sixth most likely class: Industrial -- Probability: 0.0
seventh most likely class: seaLake -- Probability: 0.0
eight most likely class: river -- Probability: 0.0
nineth most likely class: Residential -- Probability: 0.0
tenth most likely class: PermanentCrop -- Probability: 0.0
